In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
base_dir = 'D:\\Academic projects\\Minor project\\Data\\New Plant Diseases Dataset(Augmented)'
training_set = train_datagen.flow_from_directory(base_dir+'/train', target_size=(224,224), batch_size=64, class_mode='categorical')
li = list(training_set.class_indices.keys())
training_set = 0

Found 70295 images belonging to 38 classes.


In [5]:
print(li)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Sp

In [2]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import numpy as np

def predict(filepath,li):
    model = load_model('D:\\Academic projects\\Minor project\\code\\cnn2\\model.h5')

    image_path = filepath
    new_img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(new_img)
    img = np.expand_dims(img, axis=0)
    img = img/255

    prediction = model.predict(img)

    d = prediction.flatten()
    j = d.max()
    for index,item in enumerate(d):
        if item == j:
            class_name = li[index]
    return class_name

In [3]:
import pandas as pd

def pest_name(input_string):
    df = pd.read_excel("D:\\Academic projects\\Minor project\\App\\pest data.xlsx")
    
    if input_string in df["Disease"].values:
        value = df.loc[df["Disease"] == input_string, "pest"].values[0]
        return value

In [5]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image


root = Tk()

root.title('Plant Disease detection and diagnosis')

frame = tk.Frame(root, bg='#45aaf2')

disease = tk.Label(frame, text='Disease:', padx=25, pady=25, font=('verdana',16), bg='#45aaf2')
disease_name = tk.Entry(frame, font=('verdana',16))
Pesticides = tk.Label(frame, text='Pesticides:', padx=25, pady=25, font=('verdana',16), bg='#45aaf2')
Pesticide_name = tk.Entry(frame, font=('verdana',16))

lbl_show_pic = tk.Label(frame, bg='#45aaf2')
btn_browse = tk.Button(frame, text='Select Image',bg='grey', fg='#ffffff', font=('verdana',16))


def selectPic():
    global img
    filename = filedialog.askopenfilename(initialdir="/test", title="Select Image", filetypes=(("jpg images","*.jpg"),("png images","*.png")))
    img = Image.open(filename)
    img = ImageTk.PhotoImage(img)
    lbl_show_pic['image'] = img

    className = predict(filename,li)
    
    if "healthy" in className:
        disease_name.insert(0, "The plant is healthy")
        Pesticide_name.insert(0, '--')
    else:
        disease_name.insert(0, className)
        pestName = pest_name(className)
        Pesticide_name.insert(0, pestName)


btn_browse['command'] = selectPic

frame.pack()

disease.grid(row=0, column=0)
disease_name.grid(row=0, column=1, padx=(0,20))
Pesticides.grid(row=1, column=0)
Pesticide_name.grid(row=1, column=1, padx=(0,20))
lbl_show_pic.grid(row=2, column=0, columnspan="2")
btn_browse.grid(row=3, column=0, columnspan="2", padx=10, pady=10)

root.mainloop()